In [2]:
import os
# Find the latest version of spark 3.2.x  from http://www.apache.org/dist/spark/ and enter as the spark version
spark_version = 'spark-3.2.2'

os.environ['SPARK_VERSION']=spark_version

# Install Spark and Java
!apt-get update
!apt-get install openjdk-8-jdk-headless -qq > /dev/null
!wget -q http://www.apache.org/dist/spark/$SPARK_VERSION/$SPARK_VERSION-bin-hadoop2.7.tgz
!tar xf $SPARK_VERSION-bin-hadoop2.7.tgz
!pip install -q findspark

# Set Environment Variables
os.environ["JAVA_HOME"] = "/usr/lib/jvm/java-8-openjdk-amd64"
os.environ["SPARK_HOME"] = f"/content/{spark_version}-bin-hadoop2.7"

# Start a SparkSession
import findspark
findspark.init()

Get:1 http://security.ubuntu.com/ubuntu bionic-security InRelease [88.7 kB]
Get:2 https://cloud.r-project.org/bin/linux/ubuntu bionic-cran40/ InRelease [3,626 B]
Hit:3 http://archive.ubuntu.com/ubuntu bionic InRelease
Ign:4 https://developer.download.nvidia.com/compute/machine-learning/repos/ubuntu1804/x86_64  InRelease
Get:5 https://developer.download.nvidia.com/compute/cuda/repos/ubuntu1804/x86_64  InRelease [1,581 B]
Hit:6 https://developer.download.nvidia.com/compute/machine-learning/repos/ubuntu1804/x86_64  Release
Hit:7 http://ppa.launchpad.net/c2d4u.team/c2d4u4.0+/ubuntu bionic InRelease
Get:8 http://archive.ubuntu.com/ubuntu bionic-updates InRelease [88.7 kB]
Hit:9 http://ppa.launchpad.net/cran/libgit2/ubuntu bionic InRelease
Get:10 http://archive.ubuntu.com/ubuntu bionic-backports InRelease [83.3 kB]
Hit:11 http://ppa.launchpad.net/deadsnakes/ppa/ubuntu bionic InRelease
Hit:12 http://ppa.launchpad.net/graphics-drivers/ppa/ubuntu bionic InRelease
Get:13 http://security.ubuntu.c

In [3]:
# Install the PostgreSQL driver in our Colab environment
!wget https://jdbc.postgresql.org/download/postgresql-42.2.9.jar

--2022-11-28 04:57:33--  https://jdbc.postgresql.org/download/postgresql-42.2.9.jar
Resolving jdbc.postgresql.org (jdbc.postgresql.org)... 72.32.157.228, 2001:4800:3e1:1::228
Connecting to jdbc.postgresql.org (jdbc.postgresql.org)|72.32.157.228|:443... connected.
HTTP request sent, awaiting response... 200 OK
Length: 914037 (893K) [application/java-archive]
Saving to: ‘postgresql-42.2.9.jar’

postgresql-42.2.9.j 100%[===================>] 892.61K  5.77MB/s    in 0.2s    

2022-11-28 04:57:33 (5.77 MB/s) - ‘postgresql-42.2.9.jar’ saved [914037/914037]



In [4]:
# Establish a Spark session and add the Postgres driver to the filepath
from pyspark.sql import SparkSession
spark = SparkSession.builder.appName("CloudETL").config("spark.driver.extraClassPath","/content/postgresql-42.2.9.jar").getOrCreate()

In [5]:
# Read the first CSV file from an S3 bucket
from pyspark import SparkFiles
url = "https://s3.amazonaws.com/amazon-reviews-pds/tsv/amazon_reviews_us_Video_Games_v1_00.tsv.gz"
spark.sparkContext.addFile(url)
video_game_df = spark.read.csv(SparkFiles.get("amazon_reviews_us_Video_Games_v1_00.tsv.gz"), sep="\t", header=True, inferSchema=True)

# Show DataFrame
video_game_df.show()

+-----------+-----------+--------------+----------+--------------+--------------------+----------------+-----------+-------------+-----------+----+-----------------+--------------------+--------------------+-----------+
|marketplace|customer_id|     review_id|product_id|product_parent|       product_title|product_category|star_rating|helpful_votes|total_votes|vine|verified_purchase|     review_headline|         review_body|review_date|
+-----------+-----------+--------------+----------+--------------+--------------------+----------------+-----------+-------------+-----------+----+-----------------+--------------------+--------------------+-----------+
|         US|   12039526| RTIS3L2M1F5SM|B001CXYMFS|     737716809|Thrustmaster T-Fl...|     Video Games|          5|            0|          0|   N|                Y|an amazing joysti...|Used this for Eli...| 2015-08-31|
|         US|    9636577| R1ZV7R40OLHKD|B00M920ND6|     569686175|Tonsee 6 buttons ...|     Video Games|          5|    

In [6]:
# Count number of rows
print(video_game_df.count())

1785997


In [7]:
# Drop incomplete rows and show updated count
drop_na = video_game_df.dropna()
print(drop_na.count())

1785886


In [8]:
# Drop duplicates and show updated count
drop_duplicates = drop_na.dropDuplicates()
print(drop_duplicates.count())

1785886


In [9]:
# Check schema
drop_duplicates.printSchema()

root
 |-- marketplace: string (nullable = true)
 |-- customer_id: integer (nullable = true)
 |-- review_id: string (nullable = true)
 |-- product_id: string (nullable = true)
 |-- product_parent: integer (nullable = true)
 |-- product_title: string (nullable = true)
 |-- product_category: string (nullable = true)
 |-- star_rating: integer (nullable = true)
 |-- helpful_votes: integer (nullable = true)
 |-- total_votes: integer (nullable = true)
 |-- vine: string (nullable = true)
 |-- verified_purchase: string (nullable = true)
 |-- review_headline: string (nullable = true)
 |-- review_body: string (nullable = true)
 |-- review_date: string (nullable = true)



In [10]:
from pyspark.sql.functions import *

In [11]:
drop_duplicates = drop_duplicates.withColumn("review_date", to_date(col('review_date'), 'yyyy-mm-dd'))

In [12]:
drop_duplicates.printSchema()

root
 |-- marketplace: string (nullable = true)
 |-- customer_id: integer (nullable = true)
 |-- review_id: string (nullable = true)
 |-- product_id: string (nullable = true)
 |-- product_parent: integer (nullable = true)
 |-- product_title: string (nullable = true)
 |-- product_category: string (nullable = true)
 |-- star_rating: integer (nullable = true)
 |-- helpful_votes: integer (nullable = true)
 |-- total_votes: integer (nullable = true)
 |-- vine: string (nullable = true)
 |-- verified_purchase: string (nullable = true)
 |-- review_headline: string (nullable = true)
 |-- review_body: string (nullable = true)
 |-- review_date: date (nullable = true)



In [13]:
# Transform dataset to fit schema
video_game_review_id_df = drop_duplicates.select(['review_id', 'customer_id', 'product_id', 'product_parent', 'review_date'])
video_game_review_id_df.show()

+--------------+-----------+----------+--------------+-----------+
|     review_id|customer_id|product_id|product_parent|review_date|
+--------------+-----------+----------+--------------+-----------+
|R10010D6B1QM7U|   26197704|B004HX5OFW|       8155204| 2013-01-28|
|R100BI61LIKH2N|   10409381|B000QW9D14|     583724133| 2013-01-28|
|R100TPJCYDYGBU|   39071377|B00D3IKP7Y|     378642975| 2013-01-31|
|R1018OO9U6UUWO|   32826337|B0094X28J0|     272196441| 2014-01-21|
|R102YIWZEIAXT9|   16385120|B000UQAUWW|     694091073| 2007-01-29|
|R1034S7FA21OI1|   14198390|B004S5PBM0|     275810674| 2011-01-06|
|R103LC8CTAHWZ6|   16965219|B002EE7OKE|     852505908| 2013-01-07|
|R103PT7S4HRIP8|   48295356|B003R7H5TC|     318359987| 2010-01-28|
|R104RCQZYPYDXS|   21497984|B000R3BNDI|     318965716| 2008-01-28|
|R105KZIXHZW63W|   22595160|B000NKFEZI|     256274221| 2013-01-04|
|R105W3ID5NHQ48|   43562425|B002I0JNK8|     930811126| 2011-01-07|
|R106J0FROF1O34|   50248806|B0050SWUTQ|     861146594| 2014-01

In [14]:
video_game_products_df = drop_duplicates.select(['product_id', 'product_title'])
video_game_products_df.show()

+----------+--------------------+
|product_id|       product_title|
+----------+--------------------+
|B004HX5OFW|Xbox 360 Slim AC ...|
|B000QW9D14|Blue Dragon - Xbo...|
|B00D3IKP7Y|        Pokemon Ruby|
|B0094X28J0|Nintendo Wii Remo...|
|B000UQAUWW|Xbox 360 Console ...|
|B004S5PBM0|           Rocksmith|
|B002EE7OKE|Sin and Punishmen...|
|B003R7H5TC|Need for Speed: H...|
|B000R3BNDI|Lego Star Wars: T...|
|B000NKFEZI|   Slingo Quest - PC|
|B002I0JNK8|Test Drive Unlimi...|
|B0050SWUTQ|        Dead Space 3|
|B0029TSTAS|Tony Hawk: Ride S...|
|B00004YC3N|  Dynasty Warriors 2|
|B0050SYY5E|Halo: Combat Evol...|
|B001CM0PR8|Animal Crossing: ...|
|B00009KO3S|Viewtiful Joe - G...|
|B000G6IP5A|Desperate Housewi...|
|B00YY3OK0A|SteelSeries Siber...|
|B001GANUSC|Emergency Room: R...|
+----------+--------------------+
only showing top 20 rows



In [21]:
video_game_products_df = video_game_products_df.dropDuplicates(['product_id', 'product_title'])
video_game_products_df.show()

+----------+--------------------+
|product_id|       product_title|
+----------+--------------------+
|B00CJ7IUI6|The Elder Scrolls...|
|B00DHF39KS|Wolfenstein: The ...|
|B00MUTAVH6|Under Night In-Bi...|
|B001AZSEUW|              Peggle|
|B00KVOVBGM|PlayStation 4 Con...|
|B00O9VGH4Y|USPRO&reg; Headph...|
|B004OQNZY4|Phineas and Ferb:...|
|B00ZLN980O|Donop seablue 2.4...|
|B002L8W5V6|Dotop Nintendo Ga...|
|B007AJZ5PY|Nyko Game Case fo...|
|B000AOEU2K|Fire Emblem: Path...|
|B000H8BW7U|Tanarus (PC) (Com...|
|B013RADQOQ|Susenstone® 2400D...|
|B00KQXKUJ2|FIFA 15 (Ultimate...|
|B006W41X2C|Turtle Beach - Ea...|
|B000KCX9M4|Grand Theft Auto:...|
|B00YT90JWC|Red Wii Mini Cons...|
|B0096KG6A8|Wii U Super Mario...|
|B00L6AVLB0|World of Tanks-X3...|
|B000IMYKQ0|Wii Nunchuk Contr...|
+----------+--------------------+
only showing top 20 rows



In [15]:
video_game_customers = drop_duplicates.groupBy('customer_id').count()
video_game_customers_df = video_game_customers.withColumnRenamed('count', 'customer_count')
video_game_customers_df.show()

+-----------+--------------+
|customer_id|customer_count|
+-----------+--------------+
|   40504830|             8|
|    3675423|             3|
|   11393715|             9|
|   14175902|            22|
|   39664150|             4|
|   14534785|             1|
|   47765241|             7|
|   33929721|             1|
|   41446882|             8|
|   52714119|             4|
|   27910865|             1|
|   31564043|            24|
|   50722423|             2|
|   26206997|             1|
|   21130353|             1|
|   28184372|             1|
|   13005988|            16|
|   13673487|             1|
|   16100357|             1|
|   16863015|             1|
+-----------+--------------+
only showing top 20 rows



In [16]:
video_game_vine_df = drop_duplicates.select(['review_id', 'star_rating', 'helpful_votes', 'total_votes', 'vine'])
video_game_vine_df.show()

+--------------+-----------+-------------+-----------+----+
|     review_id|star_rating|helpful_votes|total_votes|vine|
+--------------+-----------+-------------+-----------+----+
|R10010D6B1QM7U|          1|            0|          0|   N|
|R100BI61LIKH2N|          5|            1|          2|   N|
|R100TPJCYDYGBU|          3|            1|          2|   N|
|R1018OO9U6UUWO|          5|            0|          0|   N|
|R102YIWZEIAXT9|          5|            2|          4|   N|
|R1034S7FA21OI1|          5|            2|          2|   N|
|R103LC8CTAHWZ6|          2|            1|          1|   N|
|R103PT7S4HRIP8|          2|            0|          0|   N|
|R104RCQZYPYDXS|          5|            7|          8|   N|
|R105KZIXHZW63W|          5|            1|          1|   N|
|R105W3ID5NHQ48|          3|            8|          9|   N|
|R106J0FROF1O34|          4|            0|          0|   N|
|R10770OX7D4UYW|          5|            0|          0|   N|
|R1077Z2P6K2RHR|          5|            

In [17]:
# Define AWS RDS parameters - Parameters deleted for submission - Please fill in appropriate values before running
my_aws_endpoint = 
my_aws_port_number = 
my_aws_database_name = 
my_aws_username = 
my_aws_password = 

In [18]:
# Define the connection string
jdbc_url=f'jdbc:postgresql://{my_aws_endpoint}:{my_aws_port_number}/{my_aws_database_name}'

# Set up the configuration parameters
config = {"user": f'{my_aws_username}', 
          "password": f'{my_aws_password}', 
          "driver":"org.postgresql.Driver"}

mode = 'append'

In [19]:
# Write the dataframe to the appropriate table in your PostgreSQL RDS
video_game_review_id_df.write.jdbc(url = jdbc_url, table = 'review_id_table', mode = mode, properties = config)

In [22]:
# Write the dataframe to the appropriate table in your PostgreSQL RDS
video_game_products_df.write.jdbc(url = jdbc_url, table = 'products', mode = mode, properties = config)

In [ ]:
# Write the dataframe to the appropriate table in your PostgreSQL RDS
video_game_customers_df.write.jdbc(url = jdbc_url, table = 'customers', mode = mode, properties = config)

In [ ]:
# Write the dataframe to the appropriate table in your PostgreSQL RDS
video_game_vine_df.write.jdbc(url = jdbc_url, table = 'vine_table', mode = mode, properties = config)